In [ ]:
library(tidyverse)

This is reference notebook made by Flaty

In [ ]:

base_path <- "/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/DataPreparation/CompData/march-machine-learning-mania-2024/"
result_regular <- read_csv(glue("{base_path}MRegularSeasonCompactResults.csv"))
seed <- read_csv(glue("{base_path}MNCAATourneySeeds.csv"))
teams <- read_csv(glue("{base_path}MTeams.csv"))

elo_rating <- read_csv("/kaggle/input/ncaa-2024/EloRating_mens_10.csv")#This data is created by the following code.(https://www.kaggle.com/code/flat831/elo-rating)

In [ ]:
create_features <- function(team_id, result_regular, seed) {
  tmp <- result_regular %>% filter(WTeamID == team_id | LTeamID == team_id)
  seed_tmp <- seed %>% filter(TeamID == team_id) %>% mutate(Seed = as.numeric(substr(Seed, 2, 3))) %>% select(-TeamID)
  
  tmp2 <- tmp %>% rename(TeamID = WTeamID, Score = WScore, Opp_TeamID = LTeamID, Opp_Score = LScore) #%>% relocate(TeamID, Score, Opp_TeamID, Opp_Score)
  tmp3 <- tmp %>% rename(TeamID = LTeamID, Score = LScore, Opp_TeamID = WTeamID, Opp_Score = WScore) #%>% relocate(TeamID, Score, Opp_TeamID, Opp_Score)
  
  tmp4 <- bind_rows(tmp2, tmp3) %>% 
    filter(TeamID == team_id) %>% 
    mutate(
      diff_score = Score - Opp_Score,
      win = ifelse(diff_score>0, 1, 0)
    ) %>% 
    arrange(Season, DayNum)
  
  tmp5 <- tmp4 %>% 
    group_by(Season) %>% 
    summarise(
      count = n(),
      win_count = sum(win),
      win_rate = sum(win)/n(),
      gap_avg = mean(diff_score)
    ) %>% 
    left_join(seed_tmp, by = "Season") %>% 
    mutate(TeamID = team_id)
  
  return(tmp5)
}

In [ ]:
res <- tibble()

for (team_id in teams$TeamID) {
  tmp <- create_features(team_id, result_regular, seed)
  res <- bind_rows(res, tmp)
}

In [ ]:
df_feat <- res %>% 
  left_join(elo_rating, by =c("TeamID", "Season")) %>%
  arrange(Season, Seed)

In [ ]:
df_feat %>% filter(Season == 2024) %>% head(16)

In [ ]:
df_feat %>% write_csv("features_mens.csv")